### 目的：爬取Momo購物網折價品訊息，以便得知目前下哪個商品最划算。
### 流程：首頁 -> 手機 -> iphone -> 折價券熱銷 -> 爬取“折價券熱銷”頁面所有商品 

In [ ]:
import requests
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
}

req = requests.get('https://www.momoshop.com.tw/category/DgrpCategory.jsp?d_code=1905200036&CPHOT=Y&flag=L',headers = headers)
#req #-->此時跳錯：ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
# -->猜測網站可能有防爬機制，以致無法順利爬取內容
# -->只要騙網站說我們是真人，不是爬蟲就好 => 在import下方加上headers,user agent資訊

#req.text   #--->顯示網頁原始碼


In [ ]:
#把原始碼丟給BeatuifulSoup解析，把解析結果丟給soup，以取得商品資訊
from bs4 import BeautifulSoup as bs
soup =  bs(req.text,'lxml')
#soup


In [ ]:
# 爬下所有資料：
goods = [] # 先設定一個空白list，用來裝爬出來的結果

for good in soup.select('.eachGood'): #從原始碼觀察到商品資訊都放在li標籤、名為"eachGood"的class裡
#     print(good)
    title = good.select_one('.prdName').get('title')
    price_r = good.select_one('.prdPrice').text.replace('$','').replace(',','')  #爬下結果立即去除多餘的字元
    price = int(price_r)  #放進資料庫前先處理資料型態
    icon = good.select_one('.iconArea').text
    goods.append({'title':title,'price':price,'icon':icon})
#     print('-'*30)  #用分隔線區分商品


In [ ]:
# 將結果放進DataFrame
import pandas as pd
df = pd.DataFrame(goods)
df.head(10)

In [ ]:
# 查看每個欄位的資料型態
df.dtypes


In [ ]:
# 依價格低到高，排序前10筆資料
df.head(10).sort_values(by = ['price'],ascending = True)


In [ ]:
# 依價格低到高，排序所有爬下的資料
df.sort_values(by = ['price'],ascending = True)


# ===== 其他 =====

In [ ]:
df.info()

### *DataFrame.astype(self: ~FrameOrSeries, dtype, copy: bool = True, errors: str = 'raise') → ~FrameOrSeries
### ---> https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html

### *Python DataFrame 欄位型態轉換方法
### ---> https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/358035/